In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import product
from plotly.offline import init_notebook_mode, iplot
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e13/sample_submission.csv
/kaggle/input/playground-series-s3e13/train.csv
/kaggle/input/playground-series-s3e13/test.csv


## Import the data

### Train data

In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e13/train.csv')
train_df.head()

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
0,0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lyme_disease
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tungiasis
2,2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Lyme_disease
3,3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,Rift_Valley_fever


In [3]:
train_df.shape

(707, 66)

### Test data

In [4]:
test_df = pd.read_csv('/kaggle/input/playground-series-s3e13/test.csv')
test_df.head()

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,lymph_swells,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash
0,707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,708,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,709,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,710,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,711,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
test_df.shape

(303, 65)

## Exploratory Data Analysis

### Check the labels distribution

## Baseline model development

### Encode the labels

In [6]:
# Encoder our prognosis labels as integers for easier decoding later
enc = OrdinalEncoder()
train_df['prognosis'] = enc.fit_transform(train_df[['prognosis']])
train_df['prognosis'][:10] # Note the values here are now ordinal (floats)

0     3.0
1     7.0
2     3.0
3    10.0
4     6.0
5     3.0
6     8.0
7     7.0
8     4.0
9     0.0
Name: prognosis, dtype: float64

In [7]:
# We split out our own test set so we can calculate an example MAP@K
X_train, X_test, y_train, y_test = train_test_split(train_df.drop('prognosis', axis=1), train_df['prognosis'], random_state=42)

In [8]:
# We'll train a simple LogisticRegression model on our training split
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)

# Normally lr_clf.predict would just predict the most likely prognosis, but with predict_proba we can get probabilities for all possible prognoses.
predictions = lr_clf.predict_proba(X_test) / 8

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [9]:
print("Output of predict_proba:")
print(predictions[0])

Output of predict_proba:
[0.01585209 0.02447828 0.03517554 0.00604367 0.009715   0.00323861
 0.00858926 0.00535092 0.00626227 0.00608648 0.00420788]


In [10]:
sorted_prediction_ids = np.argsort(-predictions,axis=1) # Note argsort sorts in ascending order, but by making all our values negative we'll end up with the highest probabilities first
print("Indices sorted by probabilities:")
print(sorted_prediction_ids)

Indices sorted by probabilities:
[[ 2  1  0 ...  7 10  5]
 [ 1  8  6 ...  9  3 10]
 [ 6  1  8 ...  3  4  0]
 ...
 [ 6  8  1 ...  9  7  0]
 [ 9  5  2 ...  7  6  0]
 [ 9  5  6 ... 10  7  0]]


In [11]:
# Now let's look at doing the above for a whole set of predictions at once:
sorted_prediction_ids = np.argsort(-predictions, axis=1)
top_3_prediction_ids = sorted_prediction_ids[:,:3]

# Because enc.inverse_transform expects a specific shape (a 2D array with 1 column) we can save the original shape to reshape to after decoding
original_shape = top_3_prediction_ids.shape
top_3_predictions = enc.inverse_transform(top_3_prediction_ids.reshape(-1, 1))
top_3_predictions = top_3_predictions.reshape(original_shape)
top_3_predictions[:10] # Spot check our first 10 values

array([['Japanese_encephalitis', 'Dengue', 'Chikungunya'],
       ['Dengue', 'West_Nile_fever', 'Rift_Valley_fever'],
       ['Rift_Valley_fever', 'Dengue', 'West_Nile_fever'],
       ['Zika', 'Yellow_Fever', 'Japanese_encephalitis'],
       ['Chikungunya', 'Tungiasis', 'Dengue'],
       ['Tungiasis', 'Dengue', 'Rift_Valley_fever'],
       ['West_Nile_fever', 'Plague', 'Zika'],
       ['Rift_Valley_fever', 'West_Nile_fever', 'Japanese_encephalitis'],
       ['Dengue', 'Chikungunya', 'Rift_Valley_fever'],
       ['Zika', 'Yellow_Fever', 'Plague']], dtype=object)

In [12]:
top_3_prediction_ids = sorted_prediction_ids[:,:3]
top_3_predictions = enc.inverse_transform(top_3_prediction_ids.reshape(-1, 1))
top_3_predictions = top_3_predictions.reshape(original_shape)# inverse_transform expects a 2D array, so we reshape our vector (but this won't be necessary when we run on multiple predictions at once)
top_3_predictions
# We got a list of 3 predictions! Great!

array([['Japanese_encephalitis', 'Dengue', 'Chikungunya'],
       ['Dengue', 'West_Nile_fever', 'Rift_Valley_fever'],
       ['Rift_Valley_fever', 'Dengue', 'West_Nile_fever'],
       ['Zika', 'Yellow_Fever', 'Japanese_encephalitis'],
       ['Chikungunya', 'Tungiasis', 'Dengue'],
       ['Tungiasis', 'Dengue', 'Rift_Valley_fever'],
       ['West_Nile_fever', 'Plague', 'Zika'],
       ['Rift_Valley_fever', 'West_Nile_fever', 'Japanese_encephalitis'],
       ['Dengue', 'Chikungunya', 'Rift_Valley_fever'],
       ['Zika', 'Yellow_Fever', 'Plague'],
       ['Zika', 'Yellow_Fever', 'Malaria'],
       ['Plague', 'Japanese_encephalitis', 'Lyme_disease'],
       ['Malaria', 'West_Nile_fever', 'Japanese_encephalitis'],
       ['Rift_Valley_fever', 'Lyme_disease', 'Plague'],
       ['Japanese_encephalitis', 'Rift_Valley_fever', 'Dengue'],
       ['Chikungunya', 'Dengue', 'West_Nile_fever'],
       ['Rift_Valley_fever', 'Dengue', 'Tungiasis'],
       ['Tungiasis', 'West_Nile_fever', 'Japanese_en

In [13]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])


In [14]:
# Our MAP@K score here is ~0.3456
mapk(y_test.values.reshape(-1, 1), top_3_prediction_ids, k=3)

0.34557438794726925

In [15]:
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)
# pred_xgb = np.zeros(df_test.shape[0])



# roc_xgb = []# list contains rmse for each fold
# n=0

# for i, (train_index, valid_index) in enumerate(kfold.split(X, y)):
#     x_train, y_train = X.iloc[train_index], y.iloc[train_index]
#     x_valid, y_valid = X.loc[valid_index], y.iloc[valid_index]
    

    
#     # XGBoost regressor
#     model_xgb = XGBClassifier(
#                           colsample_bytree = 0.5, 
#                           gamma = 1, 
#                           learning_rate = 0.01, 
#                           max_depth = 4, 
#                           min_child_weight = 8, 
#                           n_estimators = 500, 
#                           subsample = 0.6, random_state=42).fit(x_train, y_train)
    
#     # Now let's look at doing the above for a whole set of predictions at once:
#     sorted_prediction_ids = np.argsort(-predictions, axis=1)
#     top_3_prediction_ids = sorted_prediction_ids[:,:3]

#     # Because enc.inverse_transform expects a specific shape (a 2D array with 1 column) we can save the original shape to reshape to after decoding
#     original_shape = top_3_prediction_ids.shape
#     top_3_predictions = enc.inverse_transform(top_3_prediction_ids.reshape(-1, 1))
#     top_3_predictions = top_3_predictions.reshape(original_shape)
    
#     pred_xgb+= model_xgb.predict_proba(df_test)/kfold.n_splits
#     val_pred = model_xgb.predict_proba(x_valid)
#     roc_xgb.append(roc_auc_score(y_valid, val_pred))
    

#     print(f"fold: {n+1} roc XGB Classifier: {roc_xgb[n]}")
#     n+=1
# print(f"final rmsle is :{np.mean(roc_xgb)}")

In [16]:
# kfold = KFold(n_splits=8, shuffle=True, random_state=42)

In [17]:
# kfold.n_splits